First, I import *all* of the libraries I need.

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split
import itertools
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.datasets import make_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.tree import DecisionTreeRegressor
import sklearn.datasets as datasets
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, auc, classification_report, roc_curve
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import validation_curve
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC  
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier

In [2]:
df = pd.read_csv('bk_data.csv', index_col=0)

In [3]:
df.drop(['BOROUGH','NEIGHBORHOOD','BUILDING_CLASS_CATEGORY',
       'TAX_CLASS_AT_PRESENT', 'BLOCK', 'LOT', 'BUILDING_CLASS_AT_PRESENT',
       'ADDRESS', 'RESIDENTIAL_UNITS',
       'COMMERCIAL_UNITS', 'TOTAL_UNITS', 'YEAR_BUILT', 
       'BUILDING_CLASS_AT_TIME_OF_SALE', 'SALE_PRICE', 'SALE_DATE',
       'zip_mean', 'above_mean'], axis=1, inplace=True)

In [4]:
print('Target Variable')
print(df.groupby(['above_bk_mean']).above_bk_mean.count())

Target Variable
above_bk_mean
0    12252
1     3178
Name: above_bk_mean, dtype: int64


For my model:

0 = below the mean,
1 = above the mean

I think for my model, if I'm someone that's looking at investing I would want to wrongly guess that it's above the mean (false positive = Type I error), because I would not want to wrongly buy a house thinking it was below the mean.
Be better at predicting the negatives!

In [5]:
df = pd.get_dummies(df, columns=['SALE_SEASON','ZIP_CODE', 'APARTMENT_NUMBER', 'TAX_CLASS_AT_TIME_OF_SALE', 'year'], drop_first=True)

In [6]:
# Create matrix of features
X = df.drop('above_bk_mean', axis = 1) # grabs everything else but target

# Create target variable
y = df['above_bk_mean'] # y is the column we're trying to predict

# Create a list of the features being used in the 
feature_cols = X.columns

Now I'm going to train test split my data 75/25

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=45)

#### Metrics used to evaluate my model:

**Classification Accuracy:** Overall, how often is the classifier correct?

I should check this against the null accuracy - how often it would be right if it always guessed yes.

**Precision**: When a positive value is predicted, how often is the prediction correct? i.e When it predicts that it is above the mean, how often is this correct?

**Recall(sensitivity)**: What proportion of the actual positives were identified correctly
- How "sensitive" is the classifier to detecting positive instances?
- Also known as "True Positive Rate" or "Recall"

**Specificity**: When the actual value is negative, how often is the prediction correct? When it *is* below the mean, how often is it right - I think this is what I want to maximize.

How "specific" (or "selective") is the classifier in predicting positive instances?

**False Positive Rate:** When the actual value is negative, how often is the prediction incorrect? - I want this to be low!